### This file now only contains the upsert and updating Pipeline


# All imports and inits

In [1]:
import pandas as pd
import glob
import concurrent.futures
import gradio as gr
import tkinter as tk
import streamlit as st
import numpy as np
import textwrap
import PyPDF2
import requests
import os
import pinecone
from langchain.document_loaders.pdf import PyPDFDirectoryLoader
from dotenv import load_dotenv

from langchain_text_splitters import RecursiveCharacterTextSplitter

from langchain.schema.document import Document

from langchain.schema import AIMessage, HumanMessage, SystemMessage

from tkinter import scrolledtext, messagebox

from transformers import AutoModel, AutoTokenizer

from typing import List, Tuple

from openai import OpenAI

import time

# from pinecone import Pinecone, ServerlessSpec

from pinecone import Pinecone, ServerlessSpec, CloudProvider, AwsRegion, VectorType


# import voyageai


# Important: Import pinecone-client properly

# Load environment variables from .env file

load_dotenv()


DATA_PATH = os.getenv("DATA_PATH")

PINECONE_API = os.getenv("PINECONE_API")

# PINECONE_ENV = os.getenv("PINECONE_ENV")

GROQ_API_KEY = os.getenv("GROQ_API_KEY")
NVIDIA_API = os.getenv("NVIDIA_API")

# NVidia Embedding import
client = OpenAI(
    api_key=NVIDIA_API,
    base_url="https://integrate.api.nvidia.com/v1",
)

# # EMBEDDING_MODEL = "llama3-405b-8192-embed"

# vo = voyageai.Client()


# PDF Parser


In [2]:
def pdf_load_documents():
    document_loader = PyPDFDirectoryLoader(DATA_PATH)
    return document_loader.load()


# documents = pdf_load_documents()
# documents


# def extract_text_from_pdf(pdf_path: str) -> str:
#     """Extract text from a PDF file."""
#     with open(pdf_path, 'r') as file:
#         pdf_reader = PyPDF2.PdfReader(file)
#         text = ""
#         for page_num in range(len(pdf_reader.pages)):
#             page = pdf_reader.pages[page_num]
#             text += page.extract_text() + "\n"
#     return text
# extract_text_from_pdf(DATA_PATH)


### Single Directory Parser

In [ ]:
def load_documents():
    """
    Load PDF and Excel files from DATA_PATH.
    Returns a list of documents with content and metadata and a list of filenames.
    """
    documents = []
    file_names = []

    # Load PDFs
    pdf_loader = PyPDFDirectoryLoader(DATA_PATH)
    pdf_docs = pdf_loader.load()
    for doc in pdf_docs:
        documents.append(
            {
                "content": doc.page_content,
                "metadata": {
                    "source": doc.metadata.get("source", "unknown"),
                    "file_type": "pdf",
                },
            }
        )
        file_names.append(doc.metadata.get("source", "unknown"))

    # Load Excel files
    excel_files = glob.glob(os.path.join(DATA_PATH, "*.xlsx"))
    for file in excel_files:
        df = pd.read_excel(file)
        headers = df.columns.tolist()
        for _, row in df.iterrows():
            content = " ".join([f"{col}: {str(row[col])}" for col in headers])
            documents.append(
                {"content": content, "metadata": {"source": file, "file_type": "excel"}}
            )
        file_names.append(file)

    if not documents:
        print(f"No PDF or Excel files found in {DATA_PATH}")
    else:
        print(f"Loaded {len(documents)} documents")

    return documents, file_names


# Load documents
documents, file_names = load_documents()

# # Print file names
# print("Files parsed:")
# for name in file_names:
#     print(f"- {name}")

# print(f"\nTotal documents loaded: {len(documents)}")


### Multiple Directory Parser

In [3]:
def load_documents():
    """
    Load PDF and Excel files from DATA_PATH and its subdirectories.
    Returns a list of documents with content and metadata, and a list of filenames.
    """
    documents = []
    file_names = []
    processed_files = []

    # Load PDFs from all subdirectories
    pdf_loader = PyPDFDirectoryLoader(DATA_PATH, recursive=True)
    pdf_docs = pdf_loader.load()
    print("PDF DOCS len: \n\n", len(pdf_docs))
    # print("PDF DOCS: \n\n",pdf_docs)
    count = 0
    for doc in pdf_docs:
        source = doc.metadata.get("source", "unknown")
        if source not in processed_files:
            documents.append(
                {
                    "content": doc.page_content,
                    "metadata": {"source": source, "file_type": "pdf"},
                }
            )
            # print(f"PARSED DATA {source}: \n",documents[-1])

            processed_files.append(source)
            file_names.append(source)
            # print(f"Parsed PDF file : {source}")
        # print("COUNT: ", count)
        count += 1
    # return

    # Load Excel files from all subdirectories
    for root, _, files in os.walk(DATA_PATH):
        for file in files:
            if file.endswith(".xlsx"):
                file_path = os.path.join(root, file)
                if file_path not in processed_files:
                    df = pd.read_excel(file_path)
                    headers = df.columns.tolist()
                    for _, row in df.iterrows():
                        content = " ".join(
                            [f"{col}: {str(row[col])}" for col in headers]
                        )
                        documents.append(
                            {
                                "content": content,
                                "metadata": {"source": file_path, "file_type": "excel"},
                            }
                        )
                        # print("PARSED DATA: \n\n\n",documents[-1])
                    processed_files.append(file_path)
                    file_names.append(file_path)
                    print(f"Parsed Excel file: {file_path}")

    if not documents:
        print(f"No PDF or Excel files found in {DATA_PATH}")
    else:
        print(f"Loaded {len(documents)} documents")
    # Print file names
    print("Files parsed:")
    for name in processed_files:
        print(f"- {name}")
    return documents, file_names


# Load documents
documents, file_names = load_documents()


print(f"\nTotal documents loaded: {len(documents)}")


PDF DOCS len: 

 0
No PDF or Excel files found in D:\Disrupt Labs\AI Coach Project\Rag-pipelines-experiments\Course 1 - Introduction to Surgical Technology
Files parsed:

Total documents loaded: 0


# Text Splitting \ Chunking using Langchain

In [ ]:
# def split_documents(documents):
#     text_splitter = RecursiveCharacterTextSplitter(
#         chunk_size=1000,
#         chunk_overlap=500,
#         length_function=len,
#         is_separator_regex=True  # considers separators like '\n\n' if true
#     )
#     # Assuming each document is a dictionary with 'content' and 'metadata'
#     docs = []
#     for doc in documents:
#         chunks = text_splitter.split_text(doc['content'])
#         for i, chunk in enumerate(chunks):
#             docs.append({
#                 "content": chunk,
#                 "metadata": {
#                     **doc['metadata'],
#                     "chunk_id": i
#                 }
#             })
#     return docs


# # Split documents into chunks
# chunks = split_documents(documents)
# # print(len(chunks))

# ------------------------- For Nvidia EMbed v2 ------------------------------------------------------------------------------

# # Initialize NV Embed's tokenizer for precise length calculation
# tokenizer = AutoTokenizer.from_pretrained("nvidia/NV-Embed-v2")


# def split_documents(documents):
#     text_splitter = RecursiveCharacterTextSplitter.from_huggingface_tokenizer(
#         tokenizer=tokenizer,
#         chunk_size=30000,          # 30k tokens (93.75% of 32k context)
#         chunk_overlap=2000,        # Balanced overlap for continuity
#         separators=[
#             '\n\n---\n\n',         # Markdown section breaks
#             '\n\n',                # Paragraphs
#             '.\n',                 # Sentence + newline
#             '. ', '! ', '? ',      # Sentence boundaries
#             '\n', '。', '．',      # Fallback separators
#             ' ',                   # Last resort
#         ],
#         keep_separator=True        # Retain separators in chunks
#     )

#     docs = []
#     for doc in documents:
#         # Pre-process to remove excessive whitespace
#         clean_content = " ".join(doc["content"].split())

#         chunks = text_splitter.split_text(clean_content)
#         for i, chunk in enumerate(chunks):
#             token_count = len(tokenizer.tokenize(chunk))
#             docs.append({
#                 "content": chunk,
#                 "metadata": {
#                     **doc["metadata"],
#                     "chunk_id": i,
#                     "token_count": token_count,
#                     # Flag if doc fits in one chunk
#                     "is_full_document": len(chunks) == 1
#                 }
#             })
#     return docs


# ----------- Testing voyager 3 large embedding model ---------------------------

# # version 1 maq
# def get_embedding(data):
#     embeddings = vo.embed(data, model="voyage-3",
#                           input_type="document").embeddings[0]
#     return embeddings

# # version 2 maq
# def get_embedding(data):
#     try:
#         embeddings = vo.embed(data, model="voyage-3",
#                               input_type="document").embeddings[0]
#         # Wait 0.5 seconds between requests to stay under 3 RPM limit
#         time.sleep(0.5)
#         return embeddings
#     except voyageai.error.RateLimitError:
#         print("Rate limit hit, waiting 1 seconds...")
#         time.sleep(1)  # Wait longer if we hit the rate limit
#         return get_embedding(data)  # Retry the request
# # def get_embedding(data):
# #     embeddings = vo.embed(data, model="voyage-3",
# #                           input_type="document").tolist()
# #     return embeddings


# # embedding_model = vo.get_tokenizer("voyage-3")

# # Function to generate embeddings without tokenization


# # def get_embedding(data):
# #     embeddings = embedding_model.encode(data).tolist()
# #     return embeddings


# # Tokenizer to count number of tokens
# tokenizer = AutoTokenizer.from_pretrained("jinaai/jina-embeddings-v2-base-en")


# def split_documents(documents):
#     # Use ~90% of the 32 k window for chunk_size, leave room for overlap
#     chunk_size = 28800   # 90% of 32 000
#     chunk_overlap = 3200   # 10% overlap

#     text_splitter = RecursiveCharacterTextSplitter.from_huggingface_tokenizer(
#         chunk_size=chunk_size,
#         chunk_overlap=chunk_overlap,
#         tokenizer=tokenizer,
#         separators=[
#             "\n\n---\n\n",  # markdown breaks
#             "\n\n",         # paragraphs
#             ".\n",          # sentence + newline
#             ". ", "! ", "? ",
#             "\n", "。", "．",
#             " ",
#         ],
#         keep_separator=True
#     )

#     docs = []
#     for doc in documents:
#         # collapse whitespace
#         clean = " ".join(doc["content"].split())
#         chunks = text_splitter.split_text(clean)
#         for i, chunk in enumerate(chunks):
#             docs.append({
#                 "content": chunk,
#                 "metadata": {
#                     **doc.get("metadata", {}),
#                     "chunk_id": i,
#                     "token_count": len(tokenizer.encode(chunk)),
#                     "is_full_document": (len(chunks) == 1)
#                 }
#             })
#     return docs


# # Example usage
# chunks = split_documents(documents)
# print(f"Total chunks: {len(chunks)}")

# # Split documents into chunks
# chunks = split_documents(documents)
# print(len(chunks))


## Text Splitting \ Chunking for llama text embed v2 via pinecone

In [ ]:
def count_tokens(text: str) -> int:
    tokenizer = AutoTokenizer.from_pretrained("jinaai/jina-embeddings-v2-base-en")
    # Encode the text into tokens
    tokens = tokenizer.encode(text)
    return len(tokens)


def split_documents(documents):
    # Each chunk is ~800-1000 tokens to leave room for metadata tokens if needed

    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=32000,  # fit comfortably within 2048 token limit
        chunk_overlap=16000,  # helps retain context between chunks
        length_function=len,  # use token length if tokenizer available
        is_separator_regex=True,  # respect newline and semantic breaks
    )

    chunks = []

    for doc in documents:
        split_texts = text_splitter.split_text(doc["content"])

        for i, chunk in enumerate(split_texts):
            chunks.append(
                {"content": chunk, "metadata": {**doc["metadata"], "chunk_id": i}}
            )
    return chunks


# Example usage:
chunks = split_documents(documents)

# print(len(chunks))
print(chunks[0])
print(
    "Tokens: ", count_tokens(chunks[0]["content"])
)  # Check if splitting looks reasonable


# Init Pinecone

In [ ]:
pc = Pinecone(api_key=PINECONE_API)
# pc = Pinecone(api_key=PINECONE_API, headers={
#     "X-Pinecone-API-Version": "2025-04"
# })

print(PINECONE_API)


# -------- Integrated Emebeddings ---------------------------
# # Create a dense index with integrated inference
# index_name = "llama-text-embed-v2"

# pc.create_index_for_model(
#     name=index_name,
#     cloud="aws",
#     region="us-east-1",
#     embed={
#         "model": "llama-text-embed-v2",
#         "dimensions": 2046,
#         "field_map": {
#             "text": "text"  # Map the record field to be embedded
#         }
#     }
# )
# index = pc.Index(index_name)


### When to Use What:
**Use Upsert:**

When you're adding new vectors or want to replace existing vectors with new data (including changing the vector values).
When you need to add a completely new document or vector.
When you want to update both the vector values and metadata.

**Use Update:**

When you're only modifying the metadata of an existing vector.
When the vector values (embeddings) themselves are correct and only extra information like text, author, or document-related metadata needs to be updated.
Summary:
Upsert: Adds or replaces both the vector values and metadata. Use when inserting or completely replacing data.
Update: Modifies the metadata without changing the vector values. Use when the vectors are correct, but metadata needs an update.
For your case, if you just want to add or update the page_content or any other metadata for existing vectors, use update. If you want to re-upload vectors with new embeddings or metadata, use upsert.









### Creating Embeddings Via AutoModel.from_pretrained('jinaai/jina-embeddings-v2-base-en'  and Upsert each to Pinecone one by one


In [ ]:
# # Connect to the index
# index = pc.Index("ai-coach-1024")

# # index = pc.Index("ai-coach-nvidia-v2")

# # embedding_model = AutoModel.from_pretrained(
# #     'jinaai/jina-embeddings-v2-base-en', trust_remote_code=True)


# # ----------- Testing Nvidia nv embed v2 embedding model (req of memory and ram too high)---------------------------

# # embedding_model = AutoModel.from_pretrained(
# #     'nvidia/NV-Embed-v2',  # Updated to NV Embed v2
# #     trust_remote_code=True
# # )


# def upsert_chunks_to_pinecone(index, chunks):
#     count = 1
#     for chunk in chunks:
#         # Ensure the chunk has the correct structure
#         content = chunk.get('content')
#         metadata = chunk.get('metadata', {})

#         # Get the embedding for the chunk
#         embedding = get_embedding(content)

#         # Add the text as part of the metadata
#         metadata['text'] = content  # Store text in metadata

#         # Create a unique vector ID for each chunk (e.g., based on count or some unique identifier)
#         vector_id = f"vec_{count}"

#         # Upsert the embedding along with its metadata
#         index.upsert(vectors=[(vector_id, embedding, metadata)])

#         print(f"Embedding {count} upserted to Pinecone with metadata")
#         count += 1

#     print(f"All {count-1} embeddings have been upserted to Pinecone")

# # maqs version 2
# # def upsert_chunks_to_pinecone(index, chunks):
# #     count = 1
# #     for chunk in chunks:
# #         try:
# #             content = chunk.get('content')
# #             metadata = chunk.get('metadata', {})

# #             embedding = get_embedding(content)
# #             metadata['text'] = content

# #             vector_id = f"vec_{count}"
# #             index.upsert(vectors=[(vector_id, embedding, metadata)])

# #             print(f"Embedding {count} upserted to Pinecone with metadata")
# #             count += 1

# #             if count % 3 == 0:  # Process in batches of 3
# #                 print("Waiting 60 seconds before next batch...")
# #                 time.sleep(60)

# #         except Exception as e:
# #             print(f"Error processing chunk {count}: {str(e)}")
# #             time.sleep(20)  # Wait before retrying
# #             continue

# #     print(f"All {count-1} embeddings have been upserted to Pinecone")


# # Example usage
# # upsert_chunks_to_pinecone(index, chunks)


### Creating Embeddings Via **nvidia/nv-embed-v1** and Upserting to pinecone

In [ ]:
# Connect to the index
# index = pc.Index("ai-coach") -- COURSE 1 ONLY
index = pc.Index("surgical-tech-complete")  # -- COMPLETE SURGICAL TECH BOOTCAMP


# embedding_model = AutoModel.from_pretrained(
#     'jinaai/jina-embeddings-v2-base-en', trust_remote_code=True)

# # Function to generate embeddings without tokenization


# def get_embedding(data):
#     embeddings = pc.inference.embed(
#     model="llama-text-embed-v2",
#     inputs=[data],
#     parameters={
#         "input_type": "passage"
#     }
# )
#     return embeddings
# print(get_embedding("hello"))

# def upsert_chunks_to_pinecone(index, chunks):
#     count = 1
#     for chunk in chunks:
#         # Ensure the chunk has the correct structure
#         content = chunk.get('content')
#         metadata = chunk.get('metadata', {})

#         # Get the embedding for the chunk
#         embedding = get_embedding(content)

#         # Add the text as part of the metadata
#         metadata['text'] = content  # Store text in metadata

#         # Create a unique vector ID for each chunk (e.g., based on count or some unique identifier)
#         vector_id = f"vec_{count}"

#         # Upsert the embedding along with its metadata
#         index.upsert(vectors=[(vector_id, embedding, metadata)])

#         print(f"Embedding {count} upserted to Pinecone with metadata")
#         count += 1

#     print(f"All {count-1} embeddings have been upserted to Pinecone")


# # Example usage
# # Assuming `index` is your Pinecone index and `chunks` is the list of chunked documents
# upsert_chunks_to_pinecone(index, chunks)

# query_embeddings = embedding_model.encode(user_query).tolist()
# query_embeddings


# ----------------------------- Updated Code ---------------------------------

# def get_embedding(texts):
#     """
#     Get embeddings for a list of texts
#     """
#     embeddings = pc.inference.embed(
#         model="llama-text-embed-v2",
#         inputs=texts,
#         parameters={
#             "input_type": "passage",
#             "dimension": 2048
#         }
#     )
#     return embeddings


def get_embedding(text="None"):
    response = client.embeddings.create(
        input=text,
        model="nvidia/nv-embed-v1",
        encoding_format="float",
        extra_body={"input_type": "query", "truncate": "NONE"},
    )

    # print(response.data[0].embedding)
    # print(count_tokens(response.data[0].embedding))
    return response.data[0].embedding


# print(get_embedding("Surgical Conscience"))


def upsert_chunks_to_pinecone(index, chunks):
    count = 0
    for chunk in chunks:
        # Ensure the chunk has the correct structure
        content = chunk.get("content")
        metadata = chunk.get("metadata", {})

        # Get the embedding for the chunk
        # embedding = get_embedding(content).data[0]['values']
        embedding = get_embedding(content)

        # Add the text as part of the metadata
        metadata["text"] = content  # Store text in metadata
        # metadata["token_count"] = count_tokens(content)

        # Create a unique vector ID for each chunk (e.g., based on count or some unique identifier)
        vector_id = f"vec_{count}"

        # Upsert the embedding along with its metadata
        index.upsert(vectors=[(vector_id, embedding, metadata)])

        count += 1
        print(f"Embedding {count} upserted to Pinecone with metadata")

    print(f"All {count} embeddings have been upserted to Pinecone")


# upsert_chunks_to_pinecone(index, chunks)


# Update Vectors Function

In [ ]:
def update_pinecone_chunks(index, chunks):
    count = 0
    for chunk in chunks:
        # Get updated embedding
        embedding = get_embedding(chunk.page_content)

        # Extract metadata and page content
        metadata = chunk.metadata
        text = chunk.page_content

        # Create a unique vector ID for each chunk (e.g., based on count or some unique identifier)
        vector_id = f"vec_{count}"

        # Update the embedding and metadata
        index.update(id=vector_id, values=embedding, set_metadata=metadata)

        count += 1
        print(f"Embedding {count} updated in Pinecone with new metadata")

    print(f"All {count} embeddings have been updated in Pinecone")


# update_pinecone_chunks(index, chunks)


Since your application is designed to answer a wide range of student queries and suggest relevant material, you want to retrieve enough content to cover different facets of a topic without overwhelming the LLM with too much information.

# Starting Point:
- A common starting point is to set top_k between **5 and 10.**
- **top_k=5:** This can work well if your curated content is highly relevant and precise, ensuring that the top 5 matches are very close to the query.
-  **top_k=10:** If you want the coach to consider a broader range of content—perhaps to provide diverse perspectives or cover a topic more comprehensively—increasing top_k to around 10 might be beneficial.

# Experiment and Adjust:
- The “best” value depends on factors such as the diversity of your content, how densely your data covers the topics, and the quality of the embedding matches. It’s a good idea to experiment with different top_k values and evaluate the quality and relevance of the responses in your specific
